In [3]:
import pandas as pd
import os

#### GET refining margin brent
url = "https://www.ecologie.gouv.fr/sites/default/files/documents/Historique%20de%20la%20marge%20brute%20de%20raffinage%20sur%20Brent%20depuis%202015%20%28moyennes%20mensuelles%29.xlsx"
df_refining = pd.read_excel(url, sheet_name=0, skiprows=1)
df_refining = df_refining[['Moyennes mensuelles', 'en $/b']].rename(columns={'Moyennes mensuelles': 'Date_monthly'})

# Keep only rows where 'Date' can be converted to a datetime (removes annual averages table)
df_refining = df_refining[pd.to_datetime(df_refining['Date_monthly'], errors='coerce', format='%Y-%m-%d').notna()].copy()
df_refining['Date_monthly'] = pd.to_datetime(df_refining['Date_monthly'])

# Convert Barrel to Liter
df_refining['en $/liter'] = (df_refining['en $/b']/ 158.987).round(5)
df_refining = df_refining.drop(columns=['en $/b'])

print('df_refining\n', df_refining.head(5))




#### GET EUR USD by CSV
df_eur_usd_monthly_avg = pd.read_csv(
    "csv/eur_usd_monthly.csv",
    parse_dates=["Date_monthly"]
)

print("df_eur_usd_monthly_avg\n", df_eur_usd_monthly_avg)



# Merge monthly EUR/USD average with refining margin data
df_merged = pd.merge(df_refining, df_eur_usd_monthly_avg, on='Date_monthly', how='left')
df_merged['refining_margin_brent_eur_liter'] = (df_merged['en $/liter']/ df_merged['EUR_USD_monthly_avg']).round(5)

# replace negative margin value to zero
df_merged['refining_margin_brent_eur_liter'] = df_merged['refining_margin_brent_eur_liter'].clip(lower=0)
df_merged = df_merged.drop(columns=['en $/liter', 'EUR_USD_monthly_avg'])
print('df_merged\n', df_merged)

os.makedirs("csv", exist_ok=True)
df_merged.to_csv("csv/refining_margin_brent_eur_liter_monthly.csv", index=False)

df_refining
   Date_monthly  en $/liter
0   2015-01-01     0.04395
1   2015-02-01     0.04411
2   2015-03-01     0.05348
3   2015-04-01     0.04488
4   2015-05-01     0.04306
df_eur_usd_monthly_avg
      EUR_USD_monthly_avg Date_monthly
0                 1.3219   2007-01-01
1                 1.3019   2007-02-01
2                 1.3170   2007-03-01
3                 1.3370   2007-04-01
4                 1.3601   2007-05-01
..                   ...          ...
225               1.1736   2025-10-01
226               1.1528   2025-11-01
227               1.1601   2025-12-01
228               1.1750   2026-01-01
229               1.1849   2026-02-01

[230 rows x 2 columns]
df_merged
     Date_monthly  refining_margin_brent_eur_liter
0     2015-01-01                          0.03633
1     2015-02-01                          0.03900
2     2015-03-01                          0.04791
3     2015-04-01                          0.04180
4     2015-05-01                          0.03840
..        